In [1]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, LSTM, Dense 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from gensim.utils import simple_preprocess
import numpy as np 
import regex as re 
import gzip

In [14]:

def read(file_path): 
    with open(file_path, 'r',encoding='utf-8') as file: 
        text = file.read() 
    sentences = [sentence.strip() for sentence in re.split( 
        r'(?<=[.!?"])\s+', text) if sentence.strip()] 
  
    return sentences 
  
file_path = 'story1.txt'
text_data = read(file_path)[:1000]
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(text_data) 
total_words = len(tokenizer.word_index) + 1
input_sequences = [] 
for line in text_data: 
    token_list = tokenizer.texts_to_sequences([line])[0] 
    for i in range(1, len(token_list)): 
        n_gram_sequence = token_list[:i+1] 
        input_sequences.append(n_gram_sequence) 

seq_size = max([len(seq) for seq in input_sequences]) 
input_sequences = np.array(pad_sequences( 
    input_sequences, maxlen=seq_size, padding='pre')) 
X, y = input_sequences[:, :-1], input_sequences[:, -1] 
y=tf.keras.utils.to_categorical(y,num_classes=total_words)


In [15]:
model=Sequential()
model.add(Embedding(total_words,10,input_length=seq_size-1))
model.add(LSTM(128))
model.add(Dense(total_words,activation='softmax'))
optimizer=tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
x_test.size

184500

In [17]:
model.fit(x_train,y_train,epochs=100,verbose=1)

110/110 [==============================] - 12s 87ms/step - loss: 6.9726 - accuracy: 0.0377


IndexError: tuple index out of range

In [18]:
import random
seed_text="i think"
next_words=10
  
for _ in range(next_words): 
    token_list = tokenizer.texts_to_sequences([seed_text])[0] 
    token_list = pad_sequences( [token_list], maxlen=seq_size-1, padding='pre') 
    predicted_probs = model.predict(token_list) 
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word 
  
print(seed_text) 

1/1 [==============================] - 0s 42ms/step
i think the the the the the the the the the the
